# 🏘️ Slum Detection Model - Google Colab Training

**UNet-based semantic segmentation for detecting informal settlements**

- Model: ResNet34 UNet
- Target: 99.67% AUC-ROC
- Training Time: ~3-6 hours on Colab GPU

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install dependencies
!pip install segmentation-models-pytorch albumentations --quiet

In [ ]:
# Clone repository
!git clone https://github.com/Akila-Wasalathilaka/Slum-detection-model-using-UNET.git
%cd Slum-detection-model-using-UNET

In [ ]:
# Check GPU availability
import torch
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024**3} GB")

In [ ]:
# Link to your Google Drive data
!ln -s /content/drive/MyDrive/data data

# Verify data access
!ls -la data/
print("Data linked successfully!")

In [ ]:
# Quick development training (5 epochs)
!python scripts/train.py --model balanced --training development --data standard

In [ ]:
# Production training (uncomment for full training)
# !python scripts/train.py --model balanced --training production --data heavy_augmentation

In [ ]:
# View results
import matplotlib.pyplot as plt
from pathlib import Path
import json

# Find latest experiment
exp_dirs = list(Path('experiments').glob('*'))
if exp_dirs:
    latest_exp = max(exp_dirs, key=lambda x: x.stat().st_mtime)
    print(f"Latest experiment: {latest_exp.name}")
    
    # Load training history
    history_file = latest_exp / 'training_history.json'
    if history_file.exists():
        with open(history_file) as f:
            history = json.load(f)
        
        # Plot training curves
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        
        # Loss
        axes[0].plot(history['train_loss'], label='Train')
        axes[0].plot(history['val_loss'], label='Val')
        axes[0].set_title('Loss')
        axes[0].legend()
        
        # IoU
        axes[1].plot(history['train_iou'], label='Train')
        axes[1].plot(history['val_iou'], label='Val')
        axes[1].set_title('IoU')
        axes[1].legend()
        
        plt.tight_layout()
        plt.show()
        
        print(f"Best Val IoU: {max(history['val_iou']):.4f}")
else:
    print("No experiments found")

In [ ]:
# Save results to Google Drive
!cp -r experiments /content/drive/MyDrive/slum_detection_results
print("Results saved to Google Drive: /MyDrive/slum_detection_results")